# Cymbal Shops StyleSearch Demo - Multimodal Hybrid Product Search on AlloyDB

This notebook deploys the backend for a demo of Hybrid Search in [AlloyDB for PostgreSQL](https://cloud.google.com/products/alloydb?e=48754805&hl=en). It combines traditional SQL with text embeddings ([`text-embedding-005`](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings)), multimodal vector embeddings ([`multimodalembedding@001`](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-multimodal-embeddings)), and full-text search ([Generalized Inverted Index](https://www.postgresql.org/docs/current/gin.html)) with [Reciprocal Rank Fusion](https://medium.com/@devalshah1619/mathematical-intuition-behind-reciprocal-rank-fusion-rrf-explained-in-2-mins-002df0cc5e2a) re-ranking for enhanced product search.

> NOTE: This notebook uses pre-prepared data to quickly spin up a demo environment. You can see the steps that were required to prepare the data in the accompanying [data prep notebook](./cymbal_shops_hybrid_search_alloydb_data_prep.ipynb).

> **IMPORTANT:** This notebook leverages Preview features in AlloyDB AI. **[Sign up for the preview](https://docs.google.com/forms/d/e/1FAIpQLSfJ9vHIJ79nI7JWBDELPFL75pDQa4XVZQ2fxShfYddW0RwmLw/viewform)** before running this notebook. 

## Basic Setup

### Define Variables

In [ ]:
# Update these variables to match your environment
project_id = "your-project"  # @param {type:"string"}
region = "your-region"  # @param {type:"string"}
alloydb_cluster = "your-alloydb-cluster"  # @param {type:"string"}
alloydb_instance = "your-alloydb-instance"  # @param {type:"string"}
alloydb_password = input("Please provide a password to be used for 'postgres' database user: ")

# Don't change values below this line.
alloydb_database = "ecom" 
database_backup_uri = "gs://pr-public-demo-data/alloydb-retail-demo/alloydb-export/ecom.sql"

### Install Dependencies

In [ ]:
! pip install --quiet google-cloud-storage \
                      google-cloud-aiplatform \
                      asyncpg \
                      google.cloud.alloydb.connector

### Authenticate to Google Cloud within Colab
In order to access your Google Cloud Project from this notebook, you will need to Authenticate as an IAM user.

In [ ]:
from google.colab import auth

auth.authenticate_user()

### Connect Your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Enable APIs for AlloyDB and Vertex AI

In [ ]:
!gcloud services enable alloydb.googleapis.com aiplatform.googleapis.com

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Define Helper Functions

#### rest_api_helper()

In [ ]:
import requests
import google.auth
import json

# Get an access token based upon the current user
creds, _ = google.auth.default()
authed_session = google.auth.transport.requests.AuthorizedSession(creds)
access_token=creds.token

if project_id:
  authed_session.headers.update({"x-goog-user-project": project_id}) # Required to workaround a project quota bug

def rest_api_helper(
    session: requests.Session,
    url: str,
    http_verb: str,
    request_body: dict = None,
    params: dict = None
  ) -> dict:
  """Calls a REST API using a pre-authenticated requests Session."""

  headers = {"Content-Type": "application/json"}

  try:

    if http_verb == "GET":
      response = session.get(url, headers=headers, params=params)
    elif http_verb == "POST":
      response = session.post(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PUT":
      response = session.put(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PATCH":
      response = session.patch(url, json=request_body, headers=headers, params=params)
    elif http_verb == "DELETE":
      response = session.delete(url, headers=headers, params=params)
    else:
      raise ValueError(f"Unknown HTTP verb: {http_verb}")

    # Raise an exception for bad status codes (4xx or 5xx)
    response.raise_for_status()

    # Check if response has content before trying to parse JSON
    if response.content:
        return response.json()
    else:
        return {} # Return empty dict for empty responses (like 204 No Content)

  except requests.exceptions.RequestException as e:
      # Catch potential requests library errors (network, timeout, etc.)
      # Log detailed error information
      print(f"Request failed: {e}")
      if e.response is not None:
          print(f"Request URL: {e.request.url}")
          print(f"Request Headers: {e.request.headers}")
          print(f"Request Body: {e.request.body}")
          print(f"Response Status: {e.response.status_code}")
          print(f"Response Text: {e.response.text}")
          # Re-raise a more specific error or a custom one
          raise RuntimeError(f"API call failed with status {e.response.status_code}: {e.response.text}") from e
      else:
          raise RuntimeError(f"API call failed: {e}") from e
  except json.JSONDecodeError as e:
      print(f"Failed to decode JSON response: {e}")
      print(f"Response Text: {response.text}")
      raise RuntimeError(f"Invalid JSON received from API: {response.text}") from e



#### run_query()

In [ ]:
# Create AlloyDB Query Helper Function
import sqlalchemy
from sqlalchemy import text, exc
import pandas as pd

async def run_query(pool, sql: str, params = None, output_as_df: bool = True):
    """Executes a SQL query or statement against the database pool.

    Handles various SQL statements:
    - SELECT/WITH: Returns results as a DataFrame (if output_as_df=True)
      or ResultProxy. Supports parameters. Does not commit.
    - EXPLAIN/EXPLAIN ANALYZE: Executes the explain, returns the query plan
      as a formatted multi-line string. Ignores output_as_df.
      Supports parameters. Does not commit.
    - INSERT/UPDATE/DELETE/CREATE/ALTER etc.: Executes the statement,
      commits the transaction, logs info, and returns the ResultProxy.
      Supports single or bulk parameters (executemany).

    Args:
      pool: An asynchronous SQLAlchemy connection pool.
      sql: A string containing the SQL query or statement template.
      params: Optional.
        - None: Execute raw SQL (Use with caution for non-SELECT/EXPLAIN).
        - dict or tuple: Parameters for a single execution.
        - list of dicts/tuples: Parameters for bulk execution (executemany).
      output_as_df (bool): If True and query is SELECT/WITH, return pandas DataFrame.
                           Ignored for EXPLAIN and non-data-returning statements.

    Returns:
      pandas.DataFrame | str | sqlalchemy.engine.Result | None:
        - DataFrame: For SELECT/WITH if output_as_df=True.
        - str: For EXPLAIN/EXPLAIN ANALYZE, containing the formatted query plan.
        - ResultProxy: For non-SELECT/WITH/EXPLAIN statements, or SELECT/WITH
                       if output_as_df=False.
        - None: If a SQLAlchemy ProgrammingError or other specific error occurs.

    Raises:
        Exception: Catches and logs `sqlalchemy.exc.ProgrammingError`, returning None.
                   May re-raise other database exceptions.

    Example Execution:
      Single SELECT:
        sql_select = "SELECT ticker, company_name from investments LIMIT 5"
        df_result = await run_query(pool, sql_select)

      Single non-SELECT - Parameterized (Safe!):
        Parameterized INSERT:
          sql_insert = "INSERT INTO investments (ticker, company_name) VALUES (:ticker, :name)"
          params_insert = {"ticker": "NEW", "name": "New Company"}
          insert_result = await run_query(pool, sql_insert, params_insert)

        Parameterized UPDATE:
          sql_update = "UPDATE products SET price = :price WHERE id = :product_id"
          params_update = {"price": 99.99, "product_id": 123}
          update_result = await run_query(pool, sql_update, params_update)

      Bulk Update:
        docs = pd.DataFrame([
            {'id': 101, 'sparse_embedding': '[0.1, 0.2]'},
            {'id': 102, 'sparse_embedding': '[0.3, 0.4]'},
            # ... more rows
        ])

        update_sql_template = '''
            UPDATE products
            SET sparse_embedding = :embedding,
                sparse_embedding_model = 'BM25'
            WHERE id = :product_id
        ''' # Using named parameters :param_name

        # Prepare list of dictionaries for params
        data_to_update = [
            {"embedding": row.sparse_embedding, "product_id": row.id}
            for row in docs.itertuples(index=False)
        ]

        if data_to_update:
          bulk_result = await run_query(pool, update_sql_template, data_to_update)
          # bulk_result is the SQLAlchemy ResultProxy

    """
    sql_lower_stripped = sql.strip().lower()
    is_select_with = sql_lower_stripped.startswith(('select', 'with'))
    is_explain = sql_lower_stripped.startswith('explain')

    # Determine if the statement is expected to return data rows or a plan
    is_data_returning = is_select_with or is_explain

    # Determine actual DataFrame output eligibility (only for SELECT/WITH)
    effective_output_as_df = output_as_df and is_select_with

    # Check if params suggest a bulk operation (for logging purposes)
    is_bulk_operation = isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], (dict, tuple, list))

    async with pool.connect() as conn:
        try:
          # Execute with or without params
          if params:
              result = await conn.execute(text(sql), params)
          else:
              # Add warning for raw SQL only if it's NOT data-returning
              #if not is_data_returning:
                  #logging.warning("Executing non-SELECT/EXPLAIN raw SQL without parameters. Ensure SQL is safe.")
              result = await conn.execute(text(sql))

          # --- Handle statements that return data or plan ---
          if is_data_returning:
              if is_explain:
                  # Fetch and format EXPLAIN output as a string
                    try:
                        plan_rows = result.fetchall()
                        # EXPLAIN output is usually text in the first column
                        query_plan = "\n".join([str(row[0]) for row in plan_rows])
                        #logging.info(f"EXPLAIN executed successfully for: {sql[:100]}...")
                        return query_plan
                    except Exception as e:
                        logging.error(f"Error fetching/formatting EXPLAIN result: {e}")
                        return None
              else: # Handle SELECT / WITH
                  if effective_output_as_df:
                      try:
                          rows = result.fetchall()
                          column_names = result.keys()
                          df = pd.DataFrame(rows, columns=column_names)
                          #logging.info(f"SELECT/WITH executed successfully, returning DataFrame for: {sql[:100]}...")
                          return df
                      except Exception as e:
                          logging.error(f"Error converting SELECT result to DataFrame: {e}")
                          logging.info(f"Returning raw ResultProxy for SELECT/WITH due to DataFrame conversion error for: {sql[:100]}...")
                          return result # Fallback to raw result
                  else:
                      # Return raw result proxy for SELECT/WITH if df output not requested
                      #logging.info(f"SELECT/WITH executed successfully, returning ResultProxy for: {sql[:100]}...")
                      return result

          # --- Handle Non-Data Returning Statements (INSERT, UPDATE, DELETE, CREATE, etc.) ---
          else:
              await conn.commit() # Commit changes ONLY for these statements
              operation_type = sql.strip().split()[0].upper()
              row_count = result.rowcount # Note: rowcount behavior varies

              if is_bulk_operation:
                  print(f"Bulk {operation_type} executed for {len(params)} items. Result rowcount: {row_count}")
              elif operation_type in ['INSERT', 'UPDATE', 'DELETE']:
                  print(f"{operation_type} statement executed successfully. {row_count} row(s) affected.")
              else: # CREATE, ALTER, etc.
                  print(f"{operation_type} statement executed successfully. Result rowcount: {row_count}")
              return result # Return the result proxy

        except exc.ProgrammingError as e:
            # Log the error with context
            logging.error(f"SQL Programming Error executing query:\nSQL: {sql[:500]}...\nParams (sample): {str(params)[:500]}...\nError: {e}")
            # Rollback might happen automatically on context exit with error, but explicit can be clearer
            # await conn.rollback() # Consider if needed based on pool/transaction settings
            return None # Return None on handled programming errors
        except Exception as e:
            # Log other unexpected errors
            logging.error(f"An unexpected error occurred during query execution:\nSQL: {sql[:500]}...\nError: {e}")
            # await conn.rollback() # Consider if needed
            raise # Re-raise unexpected errors



## Create an AlloyDB Cluster

You will need an AlloyDB for PostgreSQL cluster to use this notebook. If you already have an AlloyDB cluster, you can skip to the `Setup Database` section. Otherwise, use the cells below to create one.

> ⏳ - Creating an AlloyDB cluster may take a few minutes.

In [ ]:
# create the AlloyDB Cluster
!gcloud beta alloydb clusters create {alloydb_cluster} --password={alloydb_password} --region={region}

# Create the AlloyDB Instance
!gcloud beta alloydb instances create {alloydb_instance} --instance-type=PRIMARY --cpu-count=2 --region={region} --cluster={alloydb_cluster}

To connect to your AlloyDB instance from this notebook, you will need to enable public IP on your instance. Alternatively, you can follow [these instructions](https://cloud.google.com/alloydb/docs/connect-external) to connect to an AlloyDB for PostgreSQL instance with Private IP from outside your VPC. You can also use the `--authorized-external-networks` flag to limit communication over public IP to specific IP address ranges if desired.

In [ ]:
# Enable Public IP on AlloyDB
!gcloud beta alloydb instances update {alloydb_instance} --region={region} --cluster={alloydb_cluster} --assign-inbound-public-ip=ASSIGN_IPV4 --database-flags="password.enforce_complexity=on"

## Setup Database

### Connect to the AlloyDB Cluster

This function will create a connection pool to your AlloyDB instance using the AlloyDB Python connector. The AlloyDB Python connector will automatically create secure connections to your AlloyDB instance using mTLS.

In [ ]:
import asyncpg

import sqlalchemy
from sqlalchemy.ext.asyncio import AsyncEngine, create_async_engine

from google.cloud.alloydb.connector import AsyncConnector, IPTypes

async def init_connection_pool(connector: AsyncConnector, db_name: str = alloydb_database, pool_size: int = 5) -> AsyncEngine:
    # initialize Connector object for connections to AlloyDB
    connection_string = f"projects/{project_id}/locations/{region}/clusters/{alloydb_cluster}/instances/{alloydb_instance}"

    async def getconn() -> asyncpg.Connection:
        conn: asyncpg.Connection = await connector.connect(
            connection_string,
            "asyncpg",
            user="postgres",
            password=alloydb_password,
            db=db_name,
            ip_type=IPTypes.PUBLIC, # # Optionally use IPTypes.PRIVATE
        )
        return conn

    pool = create_async_engine(
        "postgresql+asyncpg://",
        async_creator=getconn,
        pool_size=pool_size,
        max_overflow=0,
        isolation_level='AUTOCOMMIT'
    )
    return pool

connector = AsyncConnector()

postgres_db_pool = await init_connection_pool(connector, "postgres")
ecom_db_pool = await init_connection_pool(connector, f"{alloydb_database}")

### Import Sample Data to AlloyDB

### Add Required Permissions

In [ ]:
project_number = ! gcloud projects describe {project_id} --format='value(projectNumber)'
project_number = project_number[0]

# These permissions are required to read from GCS for the data import and to integrate with Vertex AI for on-the-fly embedding generation.
roles_array = [
    "roles/storage.admin",
    "roles/aiplatform.user"
]

for r in roles_array:
  ! gcloud projects add-iam-policy-binding {project_id} \
      --member="serviceAccount:service-{project_number}@gcp-sa-alloydb.iam.gserviceaccount.com" \
      --role="{r}"


### Create Database

In [ ]:
# Create the database
sql = f"CREATE DATABASE {alloydb_database};"
await run_query(postgres_db_pool, sql)

### Install Pre-requisite Extensions

In [ ]:
sql_array = []

sql_array.append("CREATE EXTENSION IF NOT EXISTS vector;")
sql_array.append("CREATE EXTENSION IF NOT EXISTS google_ml_integration;")
sql_array.append("CREATE EXTENSION IF NOT EXISTS alloydb_scann;")

for sql in sql_array:
  await run_query(ecom_db_pool, sql)

### Create Agentspace User

This step creates a user for an options Agentspace integration. It is required for a successful import in the next step.

In [ ]:
agentspace_user_password = input("Please provide a password to be used for 'agentspace_user' database user: ")
sql = f"CREATE ROLE agentspace_user WITH LOGIN PASSWORD '{agentspace_user_password}';"
await run_query(ecom_db_pool, sql)

### Run the Import

In [ ]:
# Reference: https://cloud.google.com/alloydb/docs/reference/rest/v1/projects.locations.clusters/import
#            https://cloud.google.com/alloydb/docs/import-sql-file

import time

url = f"https://alloydb.googleapis.com/v1/projects/{project_id}/locations/{region}/clusters/{alloydb_cluster}:import"
request_body = {
   "gcsUri": f"{database_backup_uri}",
   "database": f"{alloydb_database}",
   "user": "postgres",
   "sqlImportOptions": {}
}

result = rest_api_helper(authed_session, url, 'POST', request_body, {})
print(f"Kicked off import: {result}")

operation_id = result['name']

operation_complete = False
while operation_complete == False:
  print(f"Import still running: {operation_id}")
  url = f"https://alloydb.googleapis.com/v1/{operation_id}"
  response = rest_api_helper(authed_session, url, 'GET', request_body, {})
  operation_complete = response['done']
  if operation_complete:
    print(f"Operation complete. Check result payload for potential errors. \nResult: {response}")
    continue
  time.sleep(5)

### Check Row Counts

In [ ]:
sql = """
SELECT 'distribution_centers' AS table_name, (SELECT COUNT(*) FROM distribution_centers) AS actual_row_count, 10 AS target_row_count
UNION ALL
SELECT 'events', (SELECT COUNT(*) FROM events), 2438862
UNION ALL
SELECT 'inventory_items', (SELECT COUNT(*) FROM inventory_items), 494254
UNION ALL
SELECT 'orders', (SELECT COUNT(*) FROM orders), 125905
UNION ALL
SELECT 'order_items', (SELECT COUNT(*) FROM order_items), 182905
UNION ALL
SELECT 'products', (SELECT COUNT(*) FROM products), 29120
UNION ALL
SELECT 'users', (SELECT COUNT(*) FROM users), 100000;
"""

await run_query(ecom_db_pool, sql)

### Test the Vertex AI Integration

In [ ]:
sql = "SELECT embedding('text-embedding-005', 'This string will be transformed into an embedding.');"
await run_query(ecom_db_pool, sql)

### Create Standard PostgreSQL Indexes for Efficient Facet Searches

In [ ]:
sql_array = []

sql_array.append("DROP INDEX IF EXISTS idx_products_brand;")
sql_array.append("CREATE INDEX idx_products_brand ON products (brand);")
sql_array.append("DROP INDEX IF EXISTS idx_products_category;")
sql_array.append("CREATE INDEX idx_products_category ON products (category);")
sql_array.append("DROP INDEX IF EXISTS idx_products_retail_price;")
sql_array.append("CREATE INDEX idx_products_retail_price ON products (retail_price);")
sql_array.append("DROP INDEX IF EXISTS idx_products_sku;")
sql_array.append("CREATE INDEX idx_products_sku ON products (sku);")

for sql in sql_array:
  await run_query(ecom_db_pool, sql)

### Create ScaNN Indexes for Efficient ANN Vector Search

In [ ]:
sql_array = []

sql_array.append("CREATE EXTENSION IF NOT EXISTS alloydb_scann")
sql_array.append("SET SESSION scann.num_leaves_to_search = 1")
sql_array.append("SET SESSION scann.pre_reordering_num_neighbors=50")
sql_array.append("DROP INDEX IF EXISTS embedding_scann")
sql_array.append("""
CREATE INDEX embedding_scann ON products
  USING scann (embedding cosine)
  WITH (num_leaves=2);
""")
sql_array.append("DROP INDEX IF EXISTS product_description_embedding_scann")
sql_array.append("""
CREATE INDEX product_description_embedding_scann ON products
  USING scann (product_description_embedding cosine)
  WITH (num_leaves=2);
""")
sql_array.append("DROP INDEX IF EXISTS product_image_embedding_scann")
sql_array.append("""
CREATE INDEX product_image_embedding_scann ON products
  USING scann (product_image_embedding cosine)
  WITH (num_leaves=2);
""")

for sql in sql_array:
  await run_query(ecom_db_pool, sql)


### Create HNSW Index for Efficient BM25 ANN Sparsevec Search

In [ ]:
sql_array = []
sql_array.append("DROP INDEX IF EXISTS sparse_embedding_hnsw")
sql_array.append("""
CREATE INDEX sparse_embedding_hnsw ON products
  USING hnsw (sparse_embedding sparsevec_ip_ops)
  WITH (m = 16, ef_construction = 64);
""")
sql_array.append("SET hnsw.ef_search = 100;") # This is necessary for better recall with sparsevec

for sql in sql_array:
  await run_query(ecom_db_pool, sql)

### Create GIN Index for Efficient Full-text Search

In [ ]:
# Ref: https://www.postgresql.org/docs/current/gin.html
sql_array = []
sql_array.append("DROP INDEX IF EXISTS products_fts_document_gin;")
sql_array.append("CREATE INDEX products_fts_document_gin ON products USING GIN (fts_document);")

for sql in sql_array:
  await run_query(ecom_db_pool, sql)

### Enable The AlloyDB Columnar Engine

References:
* https://cloud.google.com/alloydb/docs/columnar-engine/configure
* https://cloud.google.com/alloydb/docs/instance-configure-database-flags#gcloud

In [ ]:
result = ! gcloud beta alloydb instances update {alloydb_instance} \
   --database-flags google_columnar_engine.enabled=on,google_columnar_engine.enable_vectorized_join=on,password.enforce_complexity=on,google_ml_integration.enable_model_support=on \
   --region={region} \
   --cluster={alloydb_cluster} \
   --project={project_id} \
   --update-mode=FORCE_APPLY

### Add Columns to Column Store Automatically

Reference: https://cloud.google.com/alloydb/docs/columnar-engine/manage-content-recommendations

In [ ]:
sql = "SELECT google_columnar_engine_recommend();"

await run_query(ecom_db_pool, sql)

### View Recommended Columns

In [ ]:
sql = "SELECT database_name, schema_name, relation_name, column_name FROM g_columnar_recommended_columns;"

await run_query(ecom_db_pool, sql)

### Validate Columns are Added to Columnar Engine

Reference: https://cloud.google.com/alloydb/docs/columnar-engine/monitor-tune

In [ ]:
sql = "SELECT * FROM g_columnar_columns;"
await run_query(ecom_db_pool, sql)

## Deploy Demo UI

Clone the [UI Repo](https://github.com/paulramsey/snippets) and follow the [instructions](https://github.com/paulramsey/snippets/tree/main/cymbal-shops-alloydb/demo_app) to deploy the demo app UI for StyleSearch.